# Linjär regression, polynomisk regression - med kod

Jämförelse mellan linjär regression och linjär regression med polynom, alltså basis function regression med x = 𝜙(x upphöjt med i). 

Inspirerad av Nhan Tran i Towards Data Science. 

In [ ]:
#importera nödvändiga bibliotek
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Importera de två dataset vi ska använda 
dataset_pos = pd.read_csv('https://s3.us-west-2.amazonaws.com/public.gamelab.fun/dataset/position_salaries.csv')
X_pos = dataset_pos.iloc[:, 1:-1].values
y_pos = dataset_pos.iloc[:, 2].values


dataset_sal = pd.read_csv('https://s3.us-west-2.amazonaws.com/public.gamelab.fun/dataset/salary_data.csv')
X_sal = dataset_sal.iloc[:, :-1].values
y_sal = dataset_sal.iloc[:, 1].values 

När nödvändiga bibliotek och det dataset vi ska använda är importerat är det dags att göra oss familjär med den data vi ska analysera. Ett bra sätt att göra det är att visualisering. Men allra först vill vi titta på de första raderna i vår data. Använd Python för att hämta dessa i kodblocket nedan. Vi vill också räkna antalet rader totalt i vårt dataset och skriva ut det, samt göra några beräkningar på data såsom medel och medianvärde på löner i respektive position.

In [ ]:
#Hur stora är våra dataset?
print ('storlek positionsdata ' + str(dataset_pos.size))
print ('antal rader position' + str(dataset_pos.shape[0]))

print ('storlek lönesdata ' + str(dataset_sal.size))
print ('antal rader lön' + str(dataset_sal.shape[0]))



In [ ]:
#Ok, posititonsdata är bara 10 rader, då kan vi titta på hela
dataset_pos


In [ ]:
#lönedata är lite större, men ändå hanterbar, vi tittar på hela den också
dataset_sal

Vad är det för typer av data i de olika tabellerna?

In [ ]:
dataset_pos.dtypes

In [ ]:
dataset_sal.dtypes

### Vi har en duplicering i ett av våra dataset, vilken kolumn är redundant?

### Är det lämpligt att använda 1,2,3..för position, varför/varför inte? Vilken typ av data är det?

In [ ]:
dataset_pos.drop(['Position'], axis=1, inplace=True)
dataset_pos

Lär känna vår data lite bättre, räkna ut medel och median för löner baserat på antal år eller efter position.

In [ ]:
mean_salary=dataset_sal['Salary'].mean()
print ('medellön baserat på arbetade år ' + str(mean_salary))

median_salary=dataset_sal['Salary'].median()
print ('medianlön baserat på arbetade år ' + str(median_salary))

In [ ]:
mean_salary=dataset_pos['Salary'].mean()
print ('medellön baserat på position ' + str(mean_salary))

median_salary=dataset_pos['Salary'].median()
print ('medianlön baserat på position ' + str(median_salary))

Inte så snyggt kanske,enklare är att från panda hämta deskriptiv data direkt om sitt dataset (Dock missar vi median då).

In [ ]:
print('deskriptiv data om positionsdata')
dataset_pos.describe()

In [ ]:
print('deskriptiv statistik om lönedata')
dataset_sal.describe()

### Är det meningsfull statistik i de olika tabellerna? Vad beskriver de?

Vi kan också skapa enkla korrelationsvisualiseringar (heatmaps) med exempelvis seabornbiblioteket. Inte så spännande när vi bara har två variabler, men ändå, vi gör först en för positionstabellen och sedan en för erfarenhetstabellen.

In [ ]:
corrMatrix = dataset_pos.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
corrMatrix = dataset_sal.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

Vi kan också skapa spridninsdiagram över relationerna mellan variablerna i våra dataset:

In [ ]:
# Plot years experience against salary

#x_sal=dataset_sal.YearsExperience.values.reshape(1,-1)
#y_sal=dataset_sal['Salary']

X_sal = dataset_sal.iloc[:, :-1]
y_sal = dataset_sal.iloc[:, 1]


# Add labels to the graph
plt.scatter(x,y)
plt.title('Salary VS Experience')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.scatter(x, y)



In [ ]:
# Plot position against salary


X_pos = dataset_pos.iloc[:, :-1]
y_pos = dataset_pos.iloc[:, 1]


#X_pos=dataset_pos['Level'].values.reshape(1,-1)
#y_pos=dataset_pos.Salary

# Add labels to the graph
plt.scatter(X_pos,y_pos)
plt.title('Salary VS Level')
plt.xlabel('Level')
plt.ylabel('Salary')
plt.scatter(X_pos, y_pos)

För att skapa både tränings- och testdata behöver vi dela vårt tillgänliga data i ett träningsset och ett testset.  Koda sedan en instruktion som delar dina två datafiler så att ca 30% blir testdata och 70% blir träningsdata. 

Ta dig sedan en funderare och reflektera över om detta är menignsfullt för dessa två tabeller och varför?

In [ ]:
# dela data mellan testdata och träningsdata för positionstabellen, är det här egentligen meningsfullt?
from sklearn.model_selection import train_test_split 
X_train_pos, X_test_pos, y_train_pos, y_test_pos = train_test_split(X_pos, y_pos, test_size=0.3)

Nu är det dags att ta en titt på en linjär regression approximering av vårt data, alltså att visualisera hur en rät linje kan anpassas till vårt data. För detta använder vi biblioteket sklearn som har metoder för linjär regression. 

In [ ]:
# Beräknar en rät linje anpassad till vårt träningdataset för positionsdata
from sklearn.linear_model import LinearRegression
lin_reg_pos = LinearRegression()
lin_reg_pos.fit(X_train_pos, y_train_pos)

#Prediktera y
y_pred_pos=lin_reg_pos.predict(X_train_pos)


# Visualisering av regressionen på träningsdata
viz_train_pos = plt
viz_train_pos.scatter(X_train_pos, y_train_pos, color='red')
viz_train_pos.plot(X_train_pos, y_pred_pos, color='blue')
viz_train_pos.title('Salary VS Position (Training set)')
viz_train_pos.xlabel('Position')
viz_train_pos.ylabel('Salary')
viz_train_pos.show()


Förutom att visualisera vårt resultat för att se hur väl vår valda modell passar vårt problem kan vi också använda beräkningar för att se hur bra vår modell är. Ett av de absolut enklaste sätten är att jämföra med medelvärdet för varje prediktkerat värde, modellen bör åtminstonde vara bättre än detta värde. 

Bättre sätt att beräkna "fit" för linjär regression är R^2 score och MSE, RMSE (Root mean square error). Dessa kan vi räkna ut direkt med hjälp av inbyggda funktioner. 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

#Beräknar kvalitetsvärden för träningsdata av positionsdata
print("mse train " + str(mean_squared_error(y_train_pos, y_pred_pos)))
print("rmse train " + str(sqrt(mean_squared_error(y_train_pos, y_pred_pos))))
print("r^2 score train "  + str(r2_score(y_train_pos, y_pred_pos)))


In [ ]:
#Dela upp dataset i testdata och träningsdata för lönetabellen
X_train_sal, X_test_sal, y_train_sal, y_test_sal = train_test_split(X_sal, y_sal, test_size=1/3, random_state=42)

In [ ]:
# Beräknar en rät linje anpassad till vårt träningsdata för lönedata
lin_reg_sal = LinearRegression()
lin_reg_sal.fit(X_train_sal, y_train_sal)


#Predikterar och visualiserar resultatet för träningsdata
y_pred_train_sal=lin_reg_sal.predict(X_train_sal)
viz_train = plt
viz_train.scatter(X_train_sal, y_train_sal, color='red')
viz_train.plot(X_train_sal, lin_reg_sal.predict(X_train_sal), color='blue')
viz_train.title('Salary VS Experience (Training set)')
viz_train.xlabel('Years of Experience')
viz_train.ylabel('Salary')
viz_train.show()

#Predikterar för testdata


#Beräknar kvalitetsvärden för träningsdata
print("mse " + str(mean_squared_error(y_train_sal, y_pred_train_sal)))
print("rmse " + str(sqrt(mean_squared_error(y_train_sal, y_pred_train_sal))))
print("r^2 score "  + str(r2_score(y_train_sal, y_pred_train_sal)))



In [ ]:
# Predikterar och visualiserar resultatet för testdata 
y_pred_test_sal=lin_reg_sal.predict(X_test_sal)
viz_test = plt
viz_test.scatter(X_test_sal, y_test_sal, color='red')
viz_test.plot(X_test_sal, y_pred_test_sal, color='blue')
viz_test.title('Salary VS Experience (Test set)')
viz_test.xlabel('Years of Experience')
viz_test.ylabel('Salary')
viz_test.show()

#Beräknar kvalitetsvärden för testsdata
print("mse " + str(mean_squared_error(y_test_sal, y_pred_test_sal)))
print("rmse " + str(sqrt(mean_squared_error(y_test_sal, y_pred_test_sal))))
print("r^2 score "  + str(r2_score(y_test_sal, y_pred_test_sal)))


### Varför gör vi inte om FIT när vi testar träningsdata?

### Märkligt att testdata är lika bra som träningsdata? En modell ska ju ha bättre FIT mot träning än mot test? (Varför?)

Ok, hur ser de ut, är en rät linje en bra approximation för våra två dataset? Vad säger våra kvalitetsvärden? 

Nja, det verkar passa utmärkt för lönedata baserat på antal år man arbetat, men betydligt sämre på lönedata baserat på position. Där behöver vi uppenbarligen testa något annat.

Vilket sätt är enklast att bestämma detta, värden eller visualisering?

Ok, då testar vi att istället anpassa en polynomiskt beräknad linje till vårt dataset med lön per position. Testa med olika grader på polynom.

OBS! Här använder jag inte test och träningsdata eftersom tabellen endast innehåller 10 rader är detta inte meningefullt och skulle inte fungera. Träningsdata skulle exempelvis bara blir 3 punkter om vi delar 70% och 30%. Samma problem finns egentligen även i lönetabellen med sina 30 rader, dock fungerar det hyfsat ändå eftersom sambandet är såpass linjärt. 

Vilken grad motsvarar vår ursprungliga linje? Vad motsvarar 0?

In [ ]:
# Istället för en rät linje approximerar vi med hjälp av polynom för positionsdata. Här med grad 1.
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=1)
X_poly = poly_reg.fit_transform(X_pos)
pol_reg_pos = LinearRegression()
pol_reg_pos.fit(X_poly, y_pos)

# Visualisera den polynomiska regressionens resultat 
def viz_polymonial():
    plt.scatter(X_pos, y_pos, color='red')
    plt.plot(X_pos, pol_reg_pos.predict(poly_reg.fit_transform(X_pos)), color='blue')
    plt.title('Polynomial grade 1')
    plt.xlabel('Position level')
    plt.ylabel('Salary')
    plt.show()
    return
viz_polymonial()

In [ ]:
# Istället för en rät linje approximerar vi med hjälp av polynom. Grad 2
poly_reg = PolynomialFeatures(degree=2)
X_pos_poly = poly_reg.fit_transform(X_pos)
pol_reg_pos = LinearRegression()
pol_reg_pos.fit(X_pos_poly, y_pos)

# Visualisera den polynomiska regressionens resultat 
y_pol_pred_pos=pol_reg_pos.predict(poly_reg.fit_transform(X_pos))
def viz_polymonial():
    plt.scatter(X_pos, y_pos, color='red')
    plt.plot(X_pos, y_pol_pred_pos, color='blue')
    plt.title('Polynomial grade 2')
    plt.xlabel('Position level')
    plt.ylabel('Salary')
    plt.show()
    return
viz_polymonial()

#Beräknar kvalitetsvärden för träningsdata med polynom
print("mse " + str(mean_squared_error(y_pos, y_pol_pred_pos)))
print("rmse " + str(sqrt(mean_squared_error(y_pos, y_pol_pred_pos))))
print("r^2 score "  + str(r2_score(y_pos, y_pol_pred_pos)))


In [ ]:
# Istället för en rät linje approximerar vi med hjälp av polynom. Grad 3
poly_reg = PolynomialFeatures(degree=3)
X_pos_poly = poly_reg.fit_transform(X_pos)
pol_reg_pos = LinearRegression()
pol_reg_pos.fit(X_pos_poly, y_pos)

# Visualisera den polynomiska regressionens resultat 
y_pol_pred_pos=pol_reg_pos.predict(poly_reg.fit_transform(X_pos))
def viz_polymonial():
    plt.scatter(X_pos, y_pos, color='red')
    plt.plot(X_pos, y_pol_pred_pos, color='blue')
    plt.title('Polynomial grade 3')
    plt.xlabel('Position level')
    plt.ylabel('Salary')
    plt.show()
    return
viz_polymonial()

#Beräknar kvalitetsvärden för träningsdata med polynom
print("mse " + str(mean_squared_error(y_pos, y_pol_pred_pos)))
print("rmse " + str(sqrt(mean_squared_error(y_pos, y_pol_pred_pos))))
print("r^2 score "  + str(r2_score(y_pos, y_pol_pred_pos)))

In [ ]:
# Istället för en rät linje approximerar vi med hjälp av polynom. Grad 4
poly_reg = PolynomialFeatures(degree=4)
X_pos_poly = poly_reg.fit_transform(X_pos)
pol_reg_pos = LinearRegression()
pol_reg_pos.fit(X_pos_poly, y_pos)

# Visualisera den polynomiska regressionens resultat 
y_pol_pred_pos=pol_reg_pos.predict(poly_reg.fit_transform(X_pos))
def viz_polymonial():
    plt.scatter(X_pos, y_pos, color='red')
    plt.plot(X_pos, y_pol_pred_pos, color='blue')
    plt.title('Polynomial grade 4')
    plt.xlabel('Position level')
    plt.ylabel('Salary')
    plt.show()
    return
viz_polymonial()

#Beräknar kvalitetsvärden för träningsdata med polynom
print("mse " + str(mean_squared_error(y_pos, y_pol_pred_pos)))
print("rmse " + str(sqrt(mean_squared_error(y_pos, y_pol_pred_pos))))
print("r^2 score "  + str(r2_score(y_pos, y_pol_pred_pos)))

In [ ]:
# Istället för en rät linje approximerar vi med hjälp av polynom. Grad 8
poly_reg = PolynomialFeatures(degree=8)
X_pos_poly = poly_reg.fit_transform(X_pos)
pol_reg_pos = LinearRegression()
pol_reg_pos.fit(X_pos_poly, y_pos)

# Visualisera den polynomiska regressionens resultat 
y_pol_pred_pos=pol_reg_pos.predict(poly_reg.fit_transform(X_pos))
def viz_polymonial():
    plt.scatter(X_pos, y_pos, color='red')
    plt.plot(X_pos, y_pol_pred_pos, color='blue')
    plt.title('Polynomial grade 8')
    plt.xlabel('Position level')
    plt.ylabel('Salary')
    plt.show()
    return
viz_polymonial()

#Beräknar kvalitetsvärden för träningsdata med polynom
print("mse " + str(mean_squared_error(y_pos, y_pol_pred_pos)))
print("rmse " + str(sqrt(mean_squared_error(y_pos, y_pol_pred_pos))))
print("r^2 score "  + str(r2_score(y_pos, y_pol_pred_pos)))

### Hur vet vi när den är overfitted?

Ok, vad ska vi nu ha detta till? Jo, för att beräkna vad för lön man ska erbjuda sina nyanställda för att de ska tacka ja! 

In [ ]:
y_pred_sal = lin_reg_sal.predict([[5.5]])
print(y_pred_sal)


In [ ]:
y_pred_pos = pol_reg_pos.predict(poly_reg.fit_transform([[7]]))
print(y_pred_pos)

In [ ]:
y_pred_pos = pol_reg_pos.predict(poly_reg.fit_transform([[7]]))
print(y_pred_pos)